In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.datasets import cifar10
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from sklearn.metrics import accuracy_score

In [ ]:
# Step 1: Load the dataset (CIFAR-10)
def load_cifar10():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize and preprocess the data for VGG16
    x_test = preprocess_input(x_test.astype(np.float32))
    y_test = tf.keras.utils.to_categorical(y_test, 10)  # One-hot encoding
    return x_test, y_test

In [ ]:
# Step 2: Load the pre-trained VGG16 model
def load_vgg16_model():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    return model

In [ ]:
# Step 3: Resize CIFAR-10 images for VGG16
def resize_images(images, target_size=(224, 224)):
    resized_images = tf.image.resize(images, target_size).numpy()
    return resized_images

In [ ]:
# Step 4: Evaluate the model on clean data
def evaluate_model(classifier, x_test, y_test):
    predictions = np.argmax(classifier.predict(x_test), axis=1)
    true_labels = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Model accuracy on clean data: {accuracy * 100:.2f}%")
    return accuracy

In [ ]:
# Step 5: Perform an FGSM attack
def perform_fgsm_attack(classifier, x_test, y_test, epsilon=0.01):
    fgsm = FastGradientMethod(estimator=classifier, eps=epsilon)
    x_adv = fgsm.generate(x=x_test)
    predictions_adv = np.argmax(classifier.predict(x_adv), axis=1)
    true_labels = np.argmax(y_test, axis=1)
    accuracy = accuracy_score(true_labels, predictions_adv)
    print(f"Model accuracy on adversarial data (FGSM, eps={epsilon}): {accuracy * 100:.2f}%")
    return x_adv, accuracy

In [ ]:
# Main workflow
def main():
    # Load data
    x_test, y_test = load_cifar10()

    # Resize images for VGG16
    x_test_resized = resize_images(x_test)

    # Load VGG16 model
    model = load_vgg16_model()

    # Wrap the model with ART's TensorFlowV2Classifier
    classifier = TensorFlowV2Classifier(
        model=model,
        nb_classes=1000,
        input_shape=(224, 224, 3),
        clip_values=(0, 255)
    )

    # Evaluate model on clean data
    evaluate_model(classifier, x_test_resized, y_test)

    # Perform FGSM attack
    perform_fgsm_attack(classifier, x_test_resized, y_test, epsilon=0.01)


In [ ]:
if __name__ == "__main__":
    main()